In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import scrapbook as sb

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.dates import date2num, DateFormatter
from pandas.plotting import scatter_matrix

plt.style.use('ggplot')
plt.tight_layout()
plt.rcParams['figure.figsize'] = [20, 20]

%matplotlib inline

In [ ]:
# Template Parameters defined here:
token = 'BTC'
from_date = '2021-12-01'
to_date = '2021-12-15'
connection_string = 'sqlite:///data/data.db'

In [ ]:
#Keeping Lookback defined to 10 periods (hours)
def vol_ohlc(df, lookback=10):
    o = df.open
    h = df.high
    l = df.low
    c = df.close
    
    k = 0.34 / (1.34 + (lookback+1)/(lookback-1))
    cc = np.log(c/c.shift(1))
    ho = np.log(h/o)
    lo = np.log(l/o)
    co = np.log(c/o)
    oc = np.log(o/c.shift(1))
    oc_sq = oc**2
    cc_sq = cc**2
    rs = ho*(ho-co)+lo*(lo-co)
    close_vol = cc_sq.rolling(lookback).sum() * (1.0 / (lookback - 1.0))
    open_vol = oc_sq.rolling(lookback).sum() * (1.0 / (lookback - 1.0))
    window_rs = rs.rolling(lookback).sum() * (1.0 / (lookback - 1.0))
    result = (open_vol + k * close_vol + (1-k) * window_rs).apply(np.sqrt) * np.sqrt(252)
    result[:lookback-1] = np.nan
    
    return result

In [ ]:
# Using same plotting function from lecture 4
def series_plot(data, events=None, ax=None):
    data.plot(ax=ax)
    start_of_days = data.between_time('00:00:00', '00:00:00').index.to_list()
    end_of_days = start_of_days[1:] + [data.index[-1]]
    colors = iter(cm.rainbow(np.linspace(0, 1, len(start_of_days))))
    for x, y in zip(start_of_days, end_of_days):
        color = next(colors)
        ax.axvspan(x, y, color=color, alpha=0.1)

In [ ]:
prices = pd.read_sql(f'''
SELECT * FROM ohlc 
WHERE token = '{token}' AND ts >= '{from_date}' 
    AND ts < '{to_date}'
ORDER BY ts
''', connection_string)
prices['ts'] = pd.to_datetime(prices['ts'])
prices.set_index('ts', inplace=True)

In [ ]:
# Perform all hourly calculations for this time series here:
prices['hourlyReturn'] = prices['close']/prices['close'].shift(1) - 1
prices['hourlyVolatility'] = vol_ohlc(prices)
# prices["peak"] = prices['close'].cummax()
# prices["hourlyDrawdown"] = (prices['close'] - prices["peak"]) / prices["peak"]
# prices["maxDrawdown"] = prices["hourlyDrawdown"].cummin()
prices["maxDrawdown"] = ((prices['close'] - prices['close'].cummax()) / prices['close'].cummax()).cummin()
prices = prices[["close", "hourlyReturn", "hourlyVolatility", "maxDrawdown", "token"]]

In [ ]:
prices

In [ ]:
%%capture
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15, 12))

In [ ]:
series_plot(prices[['hourlyReturn']], ax=ax[0, 0])
series_plot(prices[['hourlyVolatility']], ax=ax[0, 1])
series_plot(prices[['maxDrawdown']], ax=ax[1, 0])
series_plot(prices[['close']], ax=ax[1, 1])
fig.suptitle(f'Market action for {token} between {from_date} and {to_date}')
fig

In [ ]:
sb.glue('token', token)
sb.glue('from_date', from_date)
sb.glue('to_date', to_date)
sb.glue('prices', prices, encoder='pandas')
sb.glue('summary_plot', fig, encoder='display', display=False)